In [1]:
from chanlun.backtesting import backtest
from chanlun.backtesting.optimize import OptimizationSetting

import pandas as pd
from chanlun.cl_utils import query_cl_chart_config
save_path = 'D:/xuangu_log/a_strategy_hs300_tupo_1m/'

c:\Users\Zhou\anaconda3\envs\chanlun\Lib\site-packages\pyfolio\pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


Chanlun-Pro 缠论数据计算
https://gitee.com/wang-student/chanlun-pro
运行环境: Python: 3.11
授权信息: A_hia_hia_hia-VIP 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 


In [2]:
from chanlun.backtesting.base import *
# 做多
# 	触发信号：
# 		价格向上突破zg后回踩zg或小幅跌破（有个百分比范围,暂定万分之5），形成三买
# 	入场信号：
# 		放量收阳线
# 		计算最后一个向上笔到最新价格区间的frvp，得到zs,zg,zd
# 		价格>zd
	 
# 	止损：
# 		收盘价低于段中枢zg
# 	止盈：
# 		收盘价低于笔中枢zs

# 价格突破zd后回踩或小幅突破zd（有个百分比范围），行程三卖，收放量阴线，
# 做空
# 	触发信号：
# 		价格向下突破zd后回踩zd或小幅突破（有个百分比范围），形成三卖
# 	入场信号：
# 		放量收阳线
# 		计算最后一个向下笔到最新价格区间的frvp，得到zs,zg,zd
# 		价格<zg
	 
# 	止损：
# 		收盘价高于段中枢zd
# 	止盈：
# 收盘价高于笔中枢zs

class my_strategy_hs300_frvp(Strategy):
    def __init__(self):
        super().__init__()

        self._max_loss_rate = -3

    def open(
        self, code, market_data: MarketDatas, poss: Dict[str, POSITION]
    ) -> List[Operation]:
        opts = []
        high_cd = market_data.get_cl_data(code, market_data.frequencys[0])
        klines_lv0 = high_cd.get_src_klines()
        last_k = high_cd.get_src_klines()[-1]
        pre_k = high_cd.get_src_klines()[-2]
        price = last_k.c
        xdzs = high_cd.get_last_xd_zs()
        if xdzs is None:
            return opts

        xd_start = xdzs.lines[0]
        xd_zs_klines_lv0 = klines_lv0[xd_start.start.k.k_index:]
        zz_xd,zd_xd,zg_xd = self.get_frvp(cd_klines=xd_zs_klines_lv0, use_close=False)
        bi = high_cd.get_bis()[-1]
        pre_bi = high_cd.get_bis()[-2]
        # 触发信号
        if (
            bi.type == "down"
            # and bi.end.done
            and abs(bi.low - zg_xd) <= zg_xd * 0.0005
            and last_k.c >= pre_k.h
        ):
            print(f"{xd_start},{xd_zs_klines_lv0[0]}")
            print(f"{last_k.date}触发,{xd_start.start.k.index},{xd_zs_klines_lv0[0].date},{xd_zs_klines_lv0[-1].date},{price},{bi.low},{zz_xd,zd_xd,zg_xd}")
            # 入场信号
            frvp_bi_start_index = pre_bi.start.k.k_index
            pos_info = {}
            pos_info["frvp_bi_start_index"] = frvp_bi_start_index
            bi_zs_klines_lv0 = klines_lv0[frvp_bi_start_index:]
            zz_bi,zd_bi,zg_bi = self.get_frvp(cd_klines=bi_zs_klines_lv0, use_close=False)
            if (
                last_k.c > last_k.o
                # and last_k.a > pre_k.a * 1.2
                and abs(price - zd_bi) <= zd_bi * 0.0015
            ):
                print(f"符合入场条件,{last_k.date},{last_k.a/pre_k.a},{price},{zd_bi}")
                #带止损入场
                loss_price = min(bi.low, zd_xd)
                opts.append(
                    Operation(
                        code=code,
                        opt="buy",
                        mmd="3buy",
                        loss_price=loss_price,
                        info=pos_info,
                        msg=f"段zg{zg_xd},笔zd{zd_bi},止损价格：{loss_price}",
                        open_uid=f"{code}_{bi.end.k.date}"
                    )
                )
        return opts

    def close(
        self, code, mmd: str, pos: POSITION, market_data: MarketDatas
    ) -> Union[Operation, None, List[Operation]]:
        if pos.balance == 0:
            return False

        high_data = market_data.get_cl_data(code, market_data.frequencys[0])
        last_k = high_data.get_klines()[-1]
        pre_k = high_data.get_klines()[-2]
        price = last_k.c
        # bi = high_data.get_bis()[-1]
        # 止盈止损检查
        loss_opt = self.check_loss(mmd, pos, price)
        if loss_opt is not None:
            return loss_opt

        opts = []
        klines_lv0 = high_data.get_src_klines()
        frvp_bi_start_index = pos.info["frvp_bi_start_index"]
        bi_zs_klines_lv0 = klines_lv0[frvp_bi_start_index:]
        zz_bi,zd_bi,zg_bi = self.get_frvp(cd_klines=bi_zs_klines_lv0, use_close=False)
        
        if (
            price < zz_bi
            and pre_k.c >= zz_bi 
            ):
            opts.append(
                Operation(
                    code,
                    "sell",
                    mmd,
                    msg=f"{last_k.date},{bi_zs_klines_lv0[0]},{bi_zs_klines_lv0[-1]},价格低于{zz_bi}，止盈",
                    key=str(last_k.date)
                )
            )
        return opts


In [3]:
# 量化配置
bt_config = {
    # 策略结果保存的文件
    # 'save_file': 'D:/xuangu_log/a_strategy_Dnbbc/a_strategy_Dnbbc.pkl',
    'save_file': save_path + save_path.split('/')[-2] + '.pkl',

    # 设置策略对象
    'strategy': my_strategy_hs300_frvp(),
    # 回测模式：signal 信号模式，固定金额开仓； trade 交易模式，按照实际金额开仓
    'mode': 'signal',
    # 市场配置，currency 数字货币  a 沪深  hk  港股  futures  期货
    'market': 'a',
    # 基准代码，用于获取回测的时间列表
    'base_code': 'SHSE.000300',
    # 回测的标的代码，2021年后符合条件股票
    # 'codes' : ['SH.000300'],
    'codes' : ['SHSE.000300'],
    # 'codes' : ['SH.600150', 'SH.600203'], 
    # 回测的周期，这里设置里，在策略中才能取到对应周期的数据
    # 'frequencys': ['5s'],
    'frequencys': ['1m'],
    # 'frequencys': ['d','1m'],
    # 回测开始的时间
    'start_datetime': '2025-01-01 09:30:00',
    # 'start_datetime': '2023-01-01 00:00:00',
    # 回测的结束时间
    'end_datetime': '2025-03-22 15:00:00',
    # mode 为 trade 生效，初始账户资金
    'init_balance': 10000000,
    # mode 为 trade 生效，交易手续费率
    'fee_rate': 0.0000,
    # mode 为 trade 生效，最大持仓数量（分仓）
    'max_pos': 10,
    # 缠论计算的配置，详见缠论配置说明
    'cl_config': query_cl_chart_config('a', 'SH.000300'),
}

# 参数优化设置
setting = OptimizationSetting()
setting.add_cl_parameter('bi_type', ['bi_type_old', 'bi_type_new'])
setting.add_cl_parameter('bi_bzh', ['bi_bzh_yes', 'bi_bzh_no'])
setting.add_cl_parameter('zs_bi_type', ['zs_type_bz', 'zs_type_dn'])
setting.add_cl_parameter('zs_xd_type', ['zs_type_bz', 'zs_type_dn'])


BT = backtest.BackTest(bt_config)
BT.can_close_today = True  # 是否可以平今
BT.can_short = True  # 是否可以做空
BT.datas.load_data_to_cache = True # 是否将所有k线数据加载到内存中，如果内存吃紧，可以设置为 False，这样每次循环都会从数据库中读取
BT.datas.load_kline_nums = 100000 # 每次 klines 方法中返回的k线数据条数
BT.datas.cl_data_kline_max_nums = 300000 # 回测过程中，如果缠论对象的数据量超过这个值，则会清空重新计算，cl_data_kline_max_nums 不能小于 load_kline_nums，否则将会不断的删除并重新计算

In [4]:
# 运行回测
BT.run(BT.frequencys[-1])
# 多进程回测，只支持信号回测模式
# BT.run_process(BT.frequencys[-1], 22)
# 保存回测结果到文件中
BT.save()
print('Done')
# 10m 57.59s

Run SHSE.000300:   0%|          | 0/12189 [00:00<?, ?it/s]

XD index: 197 type: up start: 2025-02-05 14:05:00+08:00 end: 2025-02-07 13:28:00+08:00 high: 3919.35 low: 3785.162 is_qk: False done: True (),index: 20499 date: 2025-02-05 14:05:00+08:00 h: 3787.357 l: 3785.162 o: 3787.1 c:3786.245 a:56871700.0
2025-02-13 13:23:00+08:00触发,16769,2025-02-05 14:05:00+08:00,2025-02-13 13:23:00+08:00,3919.41,3918.006,(3882.0, 3871.13, 3919.87)
XD index: 197 type: up start: 2025-02-05 14:05:00+08:00 end: 2025-02-07 13:28:00+08:00 high: 3919.35 low: 3785.162 is_qk: False done: True (),index: 20499 date: 2025-02-05 14:05:00+08:00 h: 3787.357 l: 3785.162 o: 3787.1 c:3786.245 a:56871700.0
2025-02-13 13:24:00+08:00触发,16769,2025-02-05 14:05:00+08:00,2025-02-13 13:24:00+08:00,3921.183,3918.006,(3882.0, 3871.13, 3919.87)
XD index: 197 type: up start: 2025-02-05 14:05:00+08:00 end: 2025-02-07 13:28:00+08:00 high: 3919.35 low: 3785.162 is_qk: False done: True (),index: 20499 date: 2025-02-05 14:05:00+08:00 h: 3787.357 l: 3785.162 o: 3787.1 c:3786.245 a:56871700.0
2025

[SHSE.000300 - 2025-02-18 10:56:00+08:00] // 3buy 做多买入（3956.267 - 200），原因： 段zg3952.17,笔zd3961.43,止损价格：3923.66


XD index: 206 type: down start: 2025-02-13 10:07:00+08:00 end: 2025-02-14 09:31:00+08:00 high: 3939.198 low: 3900.827 is_qk: False done: True (段内中枢线段超过11),index: 21785 date: 2025-02-13 10:07:00+08:00 h: 3931.49 l: 3928.209 o: 3928.375 c:3931.49 a:96905900.0
2025-02-18 10:56:00+08:00触发,17808,2025-02-13 10:07:00+08:00,2025-02-18 10:56:00+08:00,3956.267,3951.645,(3938.0, 3923.66, 3952.17)
符合入场条件,2025-02-18 10:56:00+08:00,1.184726443354377,3956.267,3961.43
XD index: 206 type: down start: 2025-02-13 10:07:00+08:00 end: 2025-02-14 09:31:00+08:00 high: 3939.198 low: 3900.827 is_qk: False done: True (段内中枢线段超过11),index: 21785 date: 2025-02-13 10:07:00+08:00 h: 3931.49 l: 3928.209 o: 3928.375 c:3931.49 a:96905900.0
2025-02-18 10:58:00+08:00触发,17808,2025-02-13 10:07:00+08:00,2025-02-18 10:58:00+08:00,3958.83,3951.645,(3938.0, 3923.66, 3952.17)
符合入场条件,2025-02-18 10:58:00+08:00,1.514623658940805,3958.83,3961.43
XD index: 206 type: down start: 2025-02-13 10:07:00+08:00 end: 2025-02-14 09:31:00+08:00

[SHSE.000300 - 2025-02-18 13:07:00+08:00] // 3buy 平仓做多（3960.591 - 200），原因： 2025-02-18 13:07:00+08:00,index: 22511 date: 2025-02-18 10:16:00+08:00 h: 3938.171 l: 3936.518 o: 3936.552 c:3938.121 a:141127900.0,index: 22592 date: 2025-02-18 13:07:00+08:00 h: 3962.099 l: 3960.092 o: 3962.099 c:3960.591 a:69601100.0,价格低于3962.0，止盈
[SHSE.000300 - 3buy] // 平仓做多 (开仓：2025-02-18 10:56:00+08:00 / 3956.267 平仓：2025-02-18 13:07:00+08:00 / 3960.591) (开仓资金：791253.3999999999 平仓资金：792118.2 手续费：507.55321956) 盈亏：357.24678044004656 (0.05%)


XD index: 242 type: down start: 2025-03-07 13:22:00+08:00 end: 2025-03-10 13:11:00+08:00 high: 3967.074 low: 3905.127 is_qk: True done: True (),index: 25714 date: 2025-03-07 13:22:00+08:00 h: 3967.074 l: 3965.152 o: 3966.925 c:3965.439 a:51825200.0
2025-03-12 10:07:00+08:00触发,20992,2025-03-07 13:22:00+08:00,2025-03-12 10:07:00+08:00,3933.885,3932.215,(3915.0, 3893.41, 3931.24)
XD index: 251 type: up start: 2025-03-13 14:07:00+08:00 end: 2025-03-14 13:06:00+08:00 high: 4014.159 low: 3897.376 is_qk: False done: True (),index: 26715 date: 2025-03-13 14:07:00+08:00 h: 3898.274 l: 3897.376 o: 3897.39 c:3898.065 a:43108400.0
2025-03-19 13:30:00+08:00触发,21803,2025-03-13 14:07:00+08:00,2025-03-19 13:30:00+08:00,4016.417,4014.92,(4001.0, 3993.23, 4013.33)


运行完成，执行时间：165.3319296836853
save to : D:/xuangu_log/a_strategy_hs300_tupo_1m/a_strategy_hs300_tupo_1m.pkl


Done


In [5]:
# 输出回测结果
BT.info()
BT.result()

2025-03-22 00:03:42
保存地址 : D:/xuangu_log/a_strategy_hs300_tupo_1m/a_strategy_hs300_tupo_1m.pkl
回测模式 【signal】市场 【a】初始资金 【10000000】 手续费率 【0.0】
策略 : <__main__.my_strategy_hs300_frvp object at 0x00000246FB891710>
基准代码 : SHSE.000300
回测代码 : ['SHSE.000300']
周期 : ['1m']
起始时间 : 2025-01-01 09:30:00 结束时间 : 2025-03-22 15:00:00
缠论配置 : {'config_use_type': 'common', 'kline_type': 'kline_default', 'kline_qk': 'none', 'fx_qj': 'fx_qj_ck', 'fx_bh': 'fx_bh_no', 'bi_type': 'bi_type_old', 'bi_bzh': 'bi_bzh_yes', 'bi_qj': 'bi_qj_ck', 'bi_fx_cgd': 'bi_fx_cgd_no', 'xd_qj': 'xd_qj_ck', 'zsd_qj': 'zsd_qj_ck', 'zs_bi_type': ['zs_type_bz'], 'zs_xd_type': ['zs_type_bz'], 'zs_qj': 'zs_qj_ck', 'zs_wzgx': 'zs_wzgx_gd', 'zs_optimize': '0', 'idx_macd_fast': '12', 'idx_macd_slow': '26', 'idx_macd_signal': '9', 'fx_qy': 'fx_qy_middle', 'xd_zs_max_lines_split': '11', 'allow_split_one_line_to_xd': '0', 'allow_bi_fx_strict': '1', 'enable_kchart_low_to_high': '0', 'bi_split_k_cross_nums': '99,0', 'fx_check_k_nums': '99', 'cl

+--------------+------+------+--------+--------+------+--------+----------+----------+----------+--------+
|    买卖点    | 成功 | 失败 |  胜率  |  盈利  | 亏损 | 净利润 | 回吐比例 | 平均盈利 | 平均亏损 | 盈亏比 |
+--------------+------+------+--------+--------+------+--------+----------+----------+----------+--------+
|   一类买点   |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0    |
|   二类买点   |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0    |
|  类二类买点  |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0    |
|   三类买点   |  1   |  0   | 100.0% | 357.25 |  0   | 357.25 |   0.0    |  357.25  |    0     |   0    |
|  类三类买点  |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0    |
|  下跌笔背驰  |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0    |
| 下跌线段背驰 |  0   |  0   |   0%   |   0    |  0   |   0    |    0     |    0     |    0     |   0

In [6]:
pos_df = BT.positions()
pos_df['open_datetime'] = pd.to_datetime(pos_df['open_datetime']).dt.tz_localize(None)
pos_df['close_datetime'] = pd.to_datetime(pos_df['close_datetime']).dt.tz_localize(None)
excel_file = save_path + save_path.split('/')[-2] + '_5s.xlsx'
print(excel_file)
pos_df.to_excel(excel_file)
# pos_df.to_csv("positions_30m_direct.csv")

D:/xuangu_log/a_strategy_hs300_tupo_1m/a_strategy_hs300_tupo_1m_5s.xlsx


In [7]:
# 显示历史持仓
#设置显示全部行，不省略
pd.set_option('display.max_rows',None)
#设置显示全部列，不省略
pd.set_option('display.max_columns',None)

pos_df = BT.positions()
pos_df = pos_df.sort_values(['profit_rate'])
pos_codes = pos_df['code'].unique()

print(len(pos_df), len(pos_df[pos_df['max_profit_rate'] >= 5]))
pos_df

1 0


,code,mmd,open_datetime,close_datetime,type,price,close_price,amount,fee,loss_price,profit,profit_rate,max_profit_rate,max_loss_rate,open_msg,close_msg,open_uid
0,SHSE.000300,3buy,2025-02-18 10:56:00+08:00,2025-02-18 13:07:00+08:00,做多,3956.267,3960.591,200,507.55322,3923.66,357.24678,0.045149,0.2314,-0.0402,"段zg3952.17,笔zd3961.43,止损价格：3923.66","2025-02-18 13:07:00+08:00,index: 22511 date: 2...",SHSE.000300_2025-02-18 10:55:00+08:00


In [8]:
# 展示图表
code = "SHSE.000300"
print(BT.positions(code))
# BT.show_charts(code, BT.frequencys[0], show_futu='macd')
# BT.show_charts(code, BT.frequencys[0])

          code   mmd             open_datetime            close_datetime type  \
0  SHSE.000300  3buy 2025-02-18 10:56:00+08:00 2025-02-18 13:07:00+08:00   做多   

      price  close_price  amount        fee  loss_price     profit  \
0  3956.267     3960.591     200  507.55322     3923.66  357.24678   

   profit_rate  max_profit_rate  max_loss_rate  \
0     0.045149           0.2314        -0.0402   

                             open_msg  \
0  段zg3952.17,笔zd3961.43,止损价格：3923.66   

                                           close_msg  \
0  2025-02-18 13:07:00+08:00,index: 22511 date: 2...   

                                open_uid  
0  SHSE.000300_2025-02-18 10:55:00+08:00  


In [9]:
# 展示图表
BT.show_charts(code, BT.frequencys[0])

In [ ]:
BT.cl_config["to_file"] = f"{save_path}+{code}_{BT.frequencys[0]}_2025-3-3zhishun.html"

In [ ]:
BT.show_charts(code, BT.frequencys[0])